In [106]:
from surianalytics.connectors import ESQueryBuilder, escape as es_escape
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import plotly.express as px
import mercury as mr
import ipywidgets as widgets
import json
from IPython.display import display
from django.utils import timezone
from datetime import timedelta, timezone as dt_tz
from django.conf import settings

try:
    # allow to use timezone
    settings.configure()
    settings.USE_TZ = True
except:
    pass

# disable insecure warning
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# Be careful with max_rows, too much rows leads to freeze browser/os
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)


# Set time range for the data
global_from_date='2021-06-16T16:33:54Z'
global_to_date='2021-06-16T19:29:09Z'

def flatten_aggregation(content, keys):
    def recurse_go(val, res, keys, i, row_res):
        for item in val.get('buckets', []):
            if i == 0:
                row_res = {}
            row_res[i] = item['key']
            if str(i + 2) in item:
                recurse_go(item[str(i + 2)], res, keys, i + 1, row_res)
            else:
                for j in range(0, i + 1):
                    res[keys[j]].append(row_res[j])
                res[keys[i + 1]].append(item['doc_count'])

    res = {}
    for key in keys:
        res[key] = []
    i = 0
    for key, val in content.get('aggregations', {}).items():
        recurse_go(val, res, keys, i, {})
    pd_struct = {}
    for key in keys:
        pd_struct[key] = res[key]
    return pd.DataFrame(pd_struct)
        


In [109]:
# Get flow top talker
builder = ESQueryBuilder()
builder.set_index('logstash-flow-*')
builder.set_page_size(0)

qfilter = 'event_type: flow'
builder.set_qfilter(qfilter)

builder.add_aggs('src_ip.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('dest_ip.keyword', order='_count', sort='desc', size=10)

builder.set_from_date(global_from_date)
builder.set_to_date(global_to_date)

r = builder.post()
content = r.json()

keys = ['Source', 'Destination', 'Count']
res = flatten_aggregation(content, keys)

df = res.sort_values('Count',ascending=False)
df

,Source,Destination,Count
0,10.6.15.119,65.60.35.141,28288
1,10.6.15.119,10.6.15.5,802
10,10.6.15.187,10.6.15.5,612
20,10.6.15.93,10.6.15.5,540
2,10.6.15.119,194.226.60.15,156
11,10.6.15.187,207.246.77.75,48
3,10.6.15.119,10.6.15.255,44
12,10.6.15.187,10.6.15.255,40
32,fe80:0000:0000:0000:1c01:b22b:e837:eee2,ff02:0000:0000:0000:0000:0000:0000:00fb,38
30,10.6.15.1,224.0.0.251,38


In [ ]:
# Cipher suite count on degraded / insecure
builder = ESQueryBuilder()
builder.set_index('logstash-http-*')
builder.set_page_size(0)

qfilter = 'event_type: http'
builder.set_qfilter(qfilter)

builder.add_aggs('src_ip.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('dest_ip.keyword', order='_count', sort='desc', size=10)
builder.add_aggs('http.hostname.keyword', order='_count', sort='desc', size=10)

builder.set_from_date(global_from_date)
builder.set_to_date(global_to_date)

r = builder.post()
content = r.json()

keys = ['Source', 'Destination', 'HTTP Host', 'Count']
res = flatten_aggregation(content, keys)

df = res.sort_values('Count',ascending=False)
df

,Source,HTTP Host,Count
0,10.6.15.119,65.60.35.141,14146
1,10.6.15.119,hadevatjulps.com,78
10,10.6.15.93,download.windowsupdate.com,33
13,10.6.15.187,download.windowsupdate.com,9
2,10.6.15.119,download.windowsupdate.com,6
14,10.6.15.187,ctldl.windowsupdate.com,6
11,10.6.15.93,2.au.download.windowsupdate.com,4
12,10.6.15.93,ctldl.windowsupdate.com,4
3,10.6.15.119,au.download.windowsupdate.com,3
4,10.6.15.119,srand04rf.ru,3
